# circRNA detection using CIRCexplorer2
https://circexplorer2.readthedocs.io/en/latest/tutorial/setup/#installation-and-setup

## Installation

Circexplorer2=2.3.6

TopHat-Bowtie compatibility:
https://tophat2-and-bowtie-compatibility.readthedocs.io/en/latest/

Required versions that are compatible with each other:
* Bowtie 1.0.0
* Bowtie2 v2.1.0
* TopHat v2.0.9

Download and install Bowtie-1.0.0

In [ ]:
wget --default-page=bowtie-1.0.0-linux-x86_64.zip https://sourceforge.net/projects/bowtie-bio/files/bowtie/1.0.0/bowtie-1.0.0-linux-x86_64.zip
unzip bowtie-1.0.0-linux-x86_64.zip
mv bowtie-1.0.0/ /data/home/students/ciora/.conda/envs/oct_env/bin/bowtie-1.0.0/
export PATH=/data/home/students/ciora/.conda/envs/oct_env/bin/bowtie-1.0.0/:$PATH

Download and install Bowtie2-.2.1.0:

In [ ]:
wget --default-page=bowtie2-2.3.4.1-linux-x86_64.zip https://sourceforge.net/projects/bowtie-bio/files/bowtie2/2.1.0/bowtie2-2.1.0-linux-x86_64.zip
unzip bowtie2-2.1.0-linux-x86_64.zip
mv bowtie2-2.1.0/ /data/home/students/ciora/.conda/envs/oct_env/bin/bowtie2-2.1.0/
export PATH=/data/home/students/ciora/.conda/envs/oct_env/bin/bowtie2-2.1.0:$PATH

In [ ]:
Download and install Tophat2-2.0.9

In [ ]:
wget https://ccb.jhu.edu/software/tophat/downloads/tophat-2.0.9.Linux_x86_64.tar.gz
tar -xvzf tophat-2.0.9.Linux_x86_64.tar.gz
mv tophat-2.0.9.Linux_x86_64/ /nfs/home/students/ciora/.conda/envs/oct_env/tophat-2.0.9/
export PATH=$PATH:/bin/tophat-2.0.9.Linux_x86_64

Install CIRCExplorer2

In [ ]:
conda install -c bioconda circexplorer2=2.3.6 --yes

## Get reference genome/files

Get reference fasta file for species mmu (mouse). Use genePredToGtf from UCSC Utilities (http://hgdownload.soe.ucsc.edu/admin/exe/)

In [ ]:
fetch_ucsc.py mm10 fa mm10.fa
fetch_ucsc.py mm10 ref mm10_ref.txt
fetch_ucsc.py mm10 kg mm10_kg.txt
cut -f2-11 mm10_ref.txt|./data/reference/genePredToGtf file stdin mm10_ref.gtf
cut -f2-11 mm10_kg.txt|./data/reference/genePredToGtf file stdin mm10_kg.gtf

## Prepare input

Prepare bowtie index:

In [ ]:
bowtie-build mm10.fa bowtie1_index/mm10
bowtie2-build mm10.fa bowtie2_index/mm10
cp mm10.fa bowtie1_index/mm10.fa
cp mm10.fa bowtie2_index/mm10.fa

## Run circRNA detection

In [ ]:
tophat2 -a 6 --microexon-search -m 2 -p 16 -G mm10_kg.gtf -o tophat_out bowtie2_index/mm10 SRR5720118_1.fastq
bamToFastq -i tophat_out/unmapped.bam -fq tophat_out/unmapped.fastq

tophat2 -o tophat_fusion -p 15 --fusion-search --keep-fasta-order --no-coverage-search bowtie2_ind
ex/mm10 tophat_out/unmapped.fastq

CIRCexplorer2 parse -t TopHat-Fusion tophat_fusion/accepted_hits.bam > CIRCexplorer2_parse.log

CIRCexplorer2 annotate -r mm10_ref.txt -g mm10.fa -b back_spliced_junction.bed -o circularRNA_known.txt > CIRCexplorer2_annotate.log

### Keep processes running

In [ ]:
screen # start process
ctrl+A ctrl+D # deattach
screen -r # resume